<a href="https://colab.research.google.com/github/marcinwolter/MachineLearning2020/blob/main/mnist_mlp_minimal_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
"""
Optuna example that optimizes a neural network classifier configuration for the
MNIST dataset using Keras.
In this example, we optimize the validation accuracy of MNIST classification using
Keras. We optimize the filter and kernel size, kernel stride and layer activation.
"""
! pip install optuna

import warnings

from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop

import optuna

In [14]:

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 20


In [15]:
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(10000, 784)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    

    units = int(trial.suggest_discrete_uniform("units1", 32,1024,1))
    dropout = trial.suggest_uniform('dropout1', .00,.50)
    act = trial.suggest_categorical('activation', ['relu','sigmoid','linear'])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    epochs = int(trial.suggest_discrete_uniform('epochs', 10,50,1))

    model = Sequential()
    model.add(Dense(units, activation=act, input_shape=(784,)))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation=act))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation=act))
    model.add(Dropout(dropout))

    model.add(Dense(CLASSES, activation="softmax"))

    # We compile our model with a sampled learning rate.
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=epochs,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]


# Find the best parameters

In [16]:


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)


[I 2020-12-09 09:04:48,391] A new study created in memory with name: no-name-3665049c-2ba6-4a42-8251-800254dbd0d8
[I 2020-12-09 09:04:53,352] Trial 0 finished with value: 0.8550000190734863 and parameters: {'units1': 220.0, 'dropout1': 0.08546294765775442, 'activation': 'sigmoid', 'lr': 0.00016673106815222444, 'epochs': 50.0}. Best is trial 0 with value: 0.8550000190734863.
[I 2020-12-09 09:04:57,177] Trial 1 finished with value: 0.5080000162124634 and parameters: {'units1': 894.0, 'dropout1': 0.2863236032647772, 'activation': 'linear', 'lr': 0.0025103057949364778, 'epochs': 31.0}. Best is trial 0 with value: 0.8550000190734863.
[I 2020-12-09 09:05:01,172] Trial 2 finished with value: 0.8650000095367432 and parameters: {'units1': 273.0, 'dropout1': 0.03460094008040915, 'activation': 'linear', 'lr': 5.467638358172129e-05, 'epochs': 39.0}. Best is trial 2 with value: 0.8650000095367432.
[I 2020-12-09 09:05:03,547] Trial 3 finished with value: 0.8799999952316284 and parameters: {'units1':

In [17]:

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.9390000104904175
  Params: 
    units1: 991.0
    dropout1: 0.08507848293860804
    activation: relu
    lr: 0.0043038457379333235
    epochs: 40.0


## Visualizing the Optimization History

In [18]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

## Visualizing High-dimensional Parameter Relationships

In [19]:
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study)

## Visualizing Parameter Relationships

In [20]:
from optuna.visualization import plot_contour

plot_contour(study)

### Selecting Parameters to Visualize

In [21]:
plot_contour(study, params=['units1', 'dropout1', 'lr','epochs'])

## Visualizing Individual Parameters

In [22]:
from optuna.visualization import plot_slice

plot_slice(study)

### Selecting Parameters to Visualize

In [23]:
plot_slice(study, params=['units1', 'dropout1'])

## Visualizing Parameter Importances

In [24]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)